In [3]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings
import json
import time
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import copy
from tmpFission import tempFissionStream, tempFissionCrashesFromStation, tempFissionWeather, tempMSearch, tempFissionCrashes, tempFissionStations
warnings.filterwarnings("ignore")

FISSION_URL = 'http://172.26.135.52:9090/'
FISSION_HEADERS = {'HOST': 'fission'}

/Users/jeffphan/PycharmProjects/cloudcompa2/.venv/lib/python3.8/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://172.26.135.52:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


{'Status': 200, 'Data': 'CAPE GRIM BAPS (COMPARISON)'}


/Users/jeffphan/PycharmProjects/cloudcompa2/.venv/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.26.135.52'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
def get_full_data(params):
    # Define search query (optional, can be empty to retrieve all documents)
    data = []
    max_retries = 3
    retry_delay = 5  # seconds
    timeout = 60  # seconds

    for _ in range(max_retries):
        try:
            res = requests.get(f"{FISSION_URL}/{params}", headers=FISSION_HEADERS, timeout=timeout)
            if res.status_code != 200:
                print(res.text)
                return None
            data = json.loads(res.text)
            return data
        except requests.exceptions.RequestException as e:
            print(f"Connection error: {e}")
            print("Retrying in 5 seconds...")
            time.sleep(retry_delay)
    
    print("Max retries exceeded. Unable to retrieve data.")
    return None

In [6]:

# resp = get_full_data('stations')
# if resp is None:
#     print("Failed to retrieve data.")
# elif resp['Status'] != 200:
#     print(f"Failed to retrieve data. Status code: {resp['Status']}")
# stations = pd.DataFrame(resp["Data"])
# stations.head()
# stations_copy = stations.copy(deep=True)

In [4]:

# resp = get_full_data('stations')
resp = tempFissionStations()
if resp is None:
    print("Failed to retrieve data.")
elif resp['Status'] != 200:
    print(f"Failed to retrieve data. Status code: {resp['Status']}")
stations = pd.json_normalize(resp["Data"])
# print(resp)
# drop unnecessary columns
stations = stations.drop(columns=['_index', '_id', '_source.created_at', '_score'])
# rename columns _source.Station ID to Station ID, _source.Station Name to Station Name, _source.location Description to location
stations = stations.rename(columns={"_source.Station ID": "Station ID", "_source.Station Name": "Station Name", "_source.location": "location"})
#convert location array to 2 columns
stations = pd.concat([stations, stations['location'].apply(pd.Series)], axis=1)
# drop location column
stations = stations.drop(columns=['location'])
# rename columns 0 to Latitude, 1 to Longitude
stations = stations.rename(columns={0: "longitude", 1: "latitude"})
stations_copy = stations.copy(deep=True)

In [80]:
stations_copy.head()

,Station ID,Station Name,longitude,latitude
0,3003,BROOME AIRPORT,122.2352,-17.9475
1,9965,BUNBURY,115.6447,-33.3567
2,39128,BUNDABERG AERO,152.3230,-24.9069
3,23000,ADELAIDE (WEST TERRACE / NGAYIRDAPIRA),138.5832,-34.9257
4,87184,BREAKWATER (GEELONG RACECOURSE),144.3765,-38.1737


In [9]:
# #loop through all stations
# size = 3000 #number of crashes to get
# radius = 15 #radius in kms
# crashes = {}
# empty_stations = []
# error_stations = []

# for index, station in stations_copy.iterrows():
#     station_id = station['Station ID']
#     # station_id = "91306"
#     crashes[station_id] = crashes.get(station_id, [])
#     # params = f"crashes/{station_id}/{size}/{radius}"
#     # resp = get_full_data(params)
#     resp = tempFissionCrashesFromStation(station_id, size, radius)
#     print(resp)
#     # if resp is None or "Data" not in resp or "Token" not in resp or "Status" not in resp:
#     #     error_stations.append(station_id)
#     #     continue
#     # elif resp["Data"] == [] or resp["Token"] == "END" or resp["Status"] != 200:
#     #     empty_stations.append(station_id)
#     #     continue
#     # else:
#     #     crashes[station_id].extend(resp["Data"])
#     #     while resp and "Token" in resp and resp["Token"] != "END":
#     #         params = f"stream/{resp['Token']}"
#     #         resp = get_full_data(params)
#     #         if resp and resp["Data"]:
#     #             crashes[station_id].extend(resp["Data"])



In [5]:
#try msearch
size = 4000 #number of crashes to get
radius = 10 #radius in kms
#get list of station ID from stations
# station_ids = stations_copy['Station ID'].tolist()
crashes = tempFissionCrashes(size)
crashes_data = crashes['Data']
if crashes is None:
    print("Failed to retrieve data.")
elif crashes['Status'] != 200:
    print(f"Failed to retrieve data. Status code: {crashes['Status']}")
else:
  while(crashes['Token'] != "END"):
      crashes = tempFissionStream(crashes['Token'])
      if crashes is None:
          print("Failed to retrieve data.")
      elif crashes['Status'] != 200:
          print(f"Failed to retrieve data. Status code: {crashes['Status']}")
      else:
          crashes_data.extend(crashes['Data'])
          print(len(crashes_data))
crashes_copy = copy.deepcopy(crashes_data)

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoAxY4eG5ublpTTVI2ZS10ODBzOTM5NGZnAAAAAAAAGBgWY0dIVjhiUC1UQXFGS3lxMFVrRUVNdxY4eG5ublpTTVI2ZS10ODBzOTM5NGZnAAAAAAAAGBkWY0dIVjhiUC1UQXFGS3lxMFVrRUVNdxY4eG5ublpTTVI2ZS10ODBzOTM5NGZnAAAAAAAAGBcWY0dIVjhiUC1UQXFGS3lxMFVrRUVNdw==', 'took': 1102, 'timed_out': False, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 72052, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'crashes', '_id': '4nZkcI8B_XhVKXBOQRnc', '_score': 1.0, '_source': {'light_condition': 'Daylight', 'crash_date': '2014-02-18T00:00:00.000+0000', 'severity': 0, 'location': [147.124730869, -41.4045889398]}}, {'_index': 'crashes', '_id': '5HZkcI8B_XhVKXBOQRnc', '_score': 1.0, '_source': {'light_condition': 'Daylight', 'crash_date': '2013-12-16T00:00:00.000+0000', 'severity': 0, 'location': [147.130197396, -41.4667074329]}}, {'_index': 'crashes', '_id': '5nZkcI8B_XhVKXBOQRnc', '_score': 1.0, '_source': {'light_cond

In [6]:

crashes_copy = copy.deepcopy(crashes_data)

In [7]:
crashes_df = pd.json_normalize(crashes_copy)
# drop unnecessary columns
crashes_df = crashes_df.drop(columns=['_index', '_id', '_score'])
# rename columns _source.light_condition to light_condition, _source.crash_date to crash_date, _source.severity to severity
crashes_df = crashes_df.rename(columns={"_source.light_condition": "light_condition", "_source.crash_date": "crash_date", "_source.severity": "severity","_source.location": "location"})
#convert crash_date to datetime DD/MM/YYYY
crashes_df['crash_date'] = pd.to_datetime(crashes_df['crash_date']).dt.strftime('%d/%m/%Y')
#drop rows with missing/magic values (severity == -1)
crashes_df = crashes_df[crashes_df['severity'] != -1]
#convert location array to 2 columns
crashes_df = pd.concat([crashes_df, crashes_df['location'].apply(pd.Series)], axis=1)
# drop location column
# crashes_df = crashes_df.drop(columns=['location'])
# rename columns 0 to Latitude, 1 to Longitude
crashes_df = crashes_df.rename(columns={0: "longitude", 1: "latitude"})


In [11]:
print(crashes_df)
#deep coppy
crashes_copy = crashes_df.copy(deep=True)

                       light_condition  crash_date  severity  \
0                             Daylight  18/02/2014         0   
1                             Daylight  16/12/2013         0   
2                             Daylight  18/02/2014         0   
3                             Daylight  20/02/2014         0   
4                             Daylight  18/02/2014         0   
...                                ...         ...       ...   
72046  Darkness (without street light)  06/12/2020         0   
72047     Darkness (with street light)  05/12/2020         0   
72048                         Daylight  06/12/2020         1   
72049     Darkness (with street light)  06/12/2020         1   
72050                         Daylight  06/12/2020         0   

                              location   longitude   latitude  
0      [147.124730869, -41.4045889398]  147.124731 -41.404589  
1      [147.130197396, -41.4667074329]  147.130197 -41.466707  
2       [147.33421874, -42.9021909124] 

In [ ]:
import concurrent.futures
#loop through all stations
size = 3000 #number of crashes to get
radius = 10 #radius in kms
crashes = {}
empty_stations = []
error_stations = []

# Define a function to process each station
def process_station(station):
    station_id = station['Station ID']
    crashes[station_id] = crashes.get(station_id, [])
    resp = tempFissionCrashesFromStation(station_id, size, radius)
    if isinstance(resp, str):
        try:
            resp = json.loads(resp)
        except json.JSONDecodeError:
            print(f"Error decoding JSON for station {station_id}")
            return
    if resp is None or "Data" not in resp or "Token" not in resp or "Status" not in resp:
        error_stations.append(station_id)
        return
    elif resp["Data"] == [] or resp["Token"] == "END" or resp["Status"] != 200:
        empty_stations.append(station_id)
        return
    else:
        crashes[station_id].extend(resp["Data"])
        while resp and "Token" in resp and resp["Token"] != "END":
            resp = tempFissionStream(token=resp['Token'])
            if isinstance(resp, str):
                try:
                    resp = json.loads(resp)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON for station {station_id}")
                    return
            if resp and resp["Data"]:
                crashes[station_id].extend(resp["Data"])

# Create a ThreadPoolExecutor with a maximum of 5 threads
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit each station to the executor
    futures = [executor.submit(process_station, station) for index, station in stations_copy.iterrows()]

    # Wait for all futures to complete
    concurrent.futures.wait(futures)

# Print the results
print(crashes)
print(empty_stations)
print(error_stations)

In [ ]:


for index, station in stations_copy.iterrows():
    station_id = station['Station ID']
    print(station_id)
    # station_id = "91306"
    crashes[station_id] = crashes.get(station_id, [])
    # params = f"crashes/{station_id}/{size}/{radius}"
    # resp = get_full_data(params)

    resp = tempFissionCrashesFromStation(station_id, size, radius)
    if isinstance(resp, str):
        try:
            resp = json.loads(resp)
        except json.JSONDecodeError:
            print(f"Error decoding JSON for station {station_id}")
            continue
    if resp is None or "Data" not in resp or "Token" not in resp or "Status" not in resp:
        error_stations.append(station_id)
        continue
    elif resp["Data"] == [] or resp["Token"] == "END" or resp["Status"] != 200:
        empty_stations.append(station_id)
        continue

    else:
        crashes[station_id].extend(resp["Data"])
        print("start")
        while resp and "Token" in resp and resp["Token"] != "END":
            # params = f"stream/{resp['Token']}"
            # resp = get_full_data(params)
            resp = tempFissionStream(token=resp['Token'])
            print(resp)
            if isinstance(resp, str):
                try:
                    resp = json.loads(resp)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON for station {station_id}")
                    continue
            if resp and resp["Data"]:
                crashes[station_id].extend(resp["Data"])



In [21]:
from geopy.distance import distance
def find_nearest_station(crash, stations, radius_km):
    crash_location = (crash['latitude'], crash['longitude'])
    for _, station in stations.iterrows():
        station_location = (station['latitude'], station['longitude'])
        if distance(crash_location, station_location).km <= radius_km:
            print(station['Station ID'])
            return station['Station ID']
    return None
  
radius_km = 40  # Define the radius in kilometers

crashes_copy['nearest_station'] = crashes_copy.apply(lambda x: find_nearest_station(x, stations_copy, radius_km), axis=1)




91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
91245
9124

In [22]:
crashes_copy_2 = crashes_copy.copy(deep=True)

In [23]:
print(crashes_copy_2)

                       light_condition  crash_date  severity  \
0                             Daylight  18/02/2014         0   
1                             Daylight  16/12/2013         0   
2                             Daylight  18/02/2014         0   
3                             Daylight  20/02/2014         0   
4                             Daylight  18/02/2014         0   
...                                ...         ...       ...   
72046  Darkness (without street light)  06/12/2020         0   
72047     Darkness (with street light)  05/12/2020         0   
72048                         Daylight  06/12/2020         1   
72049     Darkness (with street light)  06/12/2020         1   
72050                         Daylight  06/12/2020         0   

                              location   longitude   latitude  nearest_station  
0      [147.124730869, -41.4045889398]  147.124731 -41.404589              NaN  
1      [147.130197396, -41.4667074329]  147.130197 -41.466707        

In [27]:

#drop na rows
crashes_copy_2 = crashes_copy_2.dropna()
print(crashes_copy_2['nearest_station'].nunique())

1


In [ ]:
#remove empty rows from crashes_copy
crashes_copy = {k: v for k, v in crashes_copy.items() if v}

#convert Dictionary to DataFrame
dfs = []
for key, value in crashes_copy.items():
  # Create a DataFrame from the list of dictionaries
    df = pd.json_normalize(value)
    #drop location and light_condition colunn
    df = df[['_source.crash_date', '_source.severity']]
    # Rename the columns
    df = df.rename(columns={'_source.crash_date': 'crash_date', '_source.severity': 'severity'})

    #format date to DD/MM/YYYY
    df['crash_date'] = pd.to_datetime(df['crash_date']).dt.strftime('%d/%m/%Y')
    
    # Add a new column 'key' with the key value
    df['Station ID'] = key
    # Add the DataFrame to the list
    dfs.append(df)

new_df = pd.concat(dfs, ignore_index=True)



In [ ]:
new_df['crash_date'] = pd.to_datetime(new_df['crash_date'], format='%d/%m/%Y')
new_df['severity'] = new_df['severity'].astype(int)
new_df['Station ID'] = new_df['Station ID'].astype(str)



In [ ]:
crashes_per_day = new_df.groupby('crash_date').size()
crashes_per_day = crashes_per_day.reset_index(name='count')
#add Station ID column
crashes_per_day['Station ID'] = station_id
crashes_per_day


In [ ]:
#find minimum year
min_date = new_df['crash_date'].min().year
#find maximum year
max_date = new_df['crash_date'].max().year
print(min_date)
print(max_date)

In [ ]:
#getting weather data from API
weather_data = []
for station in new_df['Station ID'].unique():
    params = f"weather/{station}/{min_date}/{max_date}"
    resp = get_full_data(params)
    if resp is None or "Data" not in resp or "Status" not in resp:
        error_stations.append(station)
        continue
    elif resp["Data"] == [] or resp["Status"] != 200:
        empty_stations.append(station)
        continue
    else:
        weather_data.extend(resp["Data"])

weather_data_copy = copy.deepcopy(weather_data)

weather_data_copy

In [ ]:

weather_data_copy = copy.deepcopy(weather_data)
#convert Dictionary to DataFrame
weather_dfs = pd.DataFrame(weather_data_copy)

weather_dfs['Station ID'] = station_id
#drop created_at, source, Station Name columns
weather_dfs = weather_dfs.drop(['created_at', 'source', 'Station Name', 'Pan-Rain'], axis=1)
#add dtypes for columns
weather_dfs['Date'] = pd.to_datetime(weather_dfs['Date'], format='%d/%m/%Y')
weather_dfs['Evapo-Rain'] = weather_dfs['Evapo-Rain'].astype(float)
weather_dfs['Rain'] = weather_dfs['Rain'].astype(float)
weather_dfs['Max Temp'] = weather_dfs['Max Temp'].astype(float)
weather_dfs['Min Temp'] = weather_dfs['Min Temp'].astype(float)
weather_dfs['Max Humid'] = weather_dfs['Max Humid'].astype(int)
weather_dfs['Min Humid'] = weather_dfs['Min Humid'].astype(int)
weather_dfs['WindSpeed'] = weather_dfs['WindSpeed'].astype(float)
weather_dfs['UV'] = weather_dfs['UV'].astype(float)
weather_dfs['Station ID'] = weather_dfs['Station ID'].astype(str)
#drop rows with missing/magic values
#drop Min Temp, Max Temp	= -999
weather_dfs = weather_dfs[weather_dfs['Min Temp'] != -999.0]
weather_dfs = weather_dfs[weather_dfs['Max Temp'] != -999.0]
#drop Rain, Evapo-Rain, Max Humid, Min Humid = -1.0
weather_dfs = weather_dfs[weather_dfs['Rain'] != -1.0]
weather_dfs = weather_dfs[weather_dfs['Evapo-Rain'] != -1.0]
weather_dfs = weather_dfs[weather_dfs['Max Humid'] != -1.0]
weather_dfs = weather_dfs[weather_dfs['Min Humid'] != -1.0]
weather_dfs


In [ ]:
merged_df = new_df.merge(weather_dfs, left_on=['Station ID', 'crash_date'], right_on=['Station ID', 'Date'], how='inner')

#drop Date and crash_date columns
merged_df = merged_df.drop(['Date', 'crash_date'], axis=1)

#perform label encoding
label_encoder = LabelEncoder()
merged_df['Station ID'] = label_encoder.fit_transform(merged_df['Station ID'])
merged_df

In [ ]:

# Select only the numerical columns
numerical_columns = merged_df.select_dtypes(include=[np.number])

# Calculate the correlation between 'severity' and the numerical columns
correlation = numerical_columns.corr()['severity'].sort_values()

# Print the correlation
print(correlation)

In [ ]:
#perform split on the data
from sklearn.model_selection import train_test_split
X = merged_df.drop('severity', axis=1)
y = merged_df['severity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#perform linear regression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse



In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)

In [ ]:
r2

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
merged_df = crashes_per_day.merge(weather_dfs, left_on=['Station ID', 'crash_date'], right_on=['Station ID', 'Date'], how='inner')
# merged_df = merged_df.merge(crashes_per_day, on=['Station ID', 'crash_date'], how='inner')
#drop Date and crash_date columns
merged_df = merged_df.drop(['Date', 'crash_date','Station ID'], axis=1)
merged_df

In [ ]:
# Select only the numerical columns
numerical_columns = merged_df.select_dtypes(include=[np.number])

# Calculate the correlation between 'severity' and the numerical columns
correlation = numerical_columns.corr()['count'].sort_values()

# Print the correlation
print(merged_df.corr())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(numerical_columns.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
from scipy import stats

# Assuming you have two sets of observations, obs1 and obs2
obs1 = [1, 2, 3, 4, 5]
obs2 = [2, 3, 4, 5, 6]

t_stat, p_value = stats.ttest_ind(obs1, obs2)

print("P-value: ", p_value)